In [1]:
# importing needed libraries for two stage least squares and ols

import pandas as pd
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS
import statsmodels.api as sm


# reading in the data
df = pd.read_csv("~/Desktop/Thesis Data/FinalCSV.csv", header=0)

# creating a new column that will represent revenue in thousands of dollars
df['ResponsiveRevThous'] = df['ResponsiveRev']/1000

# making the year a categorical variable
df.Year.astype('category')

# setting the index to program and year. This makes python read the data as a panel
df = df.set_index(['Program','Year'])

In [2]:
var = df

var = var.drop(columns=['ResponsiveRev','srs','new_coach'])

var.head()

TwoStars  ThreeStars  FourStars  FiveStars  lagged_srs  \
Program Year                                                           
Alabama 2010         6          38         58          8       23.69   
        2011         3          32         62         11       18.31   
        2012         1          33         57         11       24.44   
        2013         1          32         56         11       24.51   
        2014         1          29         54         16       20.06   

              stateunemp  stadium_cap  stadium_years_open  \
Program Year                                                
Alabama 2010       10.54        92138                  81   
        2011        9.62       101821                  82   
        2012        7.98        92138                  83   
        2013        7.22       101821                  84   
        2014        6.77       101821                  85   

              stadium_years_rennovation  num_homegames         ...          \
Program Year                                                   ...           
Alabama 2010                          0              7         ...           
        2011                          1              7         ...           
        2012                          2              7         ...           
        2013                          3              7         ...           
        2014                          4              7         ...           

              conf_coy  nat_coy  coach_years_at_school  pop/d1prog  \
Program Year                                                         
Alabama 2010         1        1                      3  1196394.75   
        2011         1        1                      4  1199662.25   
        2012         1        1                      5   962789.20   
        2013         1        1                      6   965532.00   
        2014         1        1                      7   968007.40   

              hs_grad_pct  pro_team_per_million  spring_practice  num_jcs  \
Program Year                                                                
Alabama 2010        86.90                   0.0                1        0   
        2011        85.20                   0.0                1        0   
        2012        86.33                   0.0                1        0   
        2013        86.59                   0.0                1        0   
        2014        86.86                   0.0                1        0   

              power  ResponsiveRevThous  
Program Year                             
Alabama 2010      1           44610.269  
        2011      1           48357.063  
        2012      1           50950.527  
        2013      1           56992.434  
        2014      1           72449.359  

[5 rows x 21 columns]

In [3]:
p5 = var[var['power'] == 1]
nonp5 =var[var['power'] == 0]

### Summary stats

In [4]:
des = var.describe()
d = des.reset_index()

d2 = d.transpose()

d2

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
TwoStars,378,33.3783,25.2668,0,12,25,58,96
ThreeStars,378,44.1376,17.6661,6,29,46,59,86
FourStars,378,14.1481,16.1375,0,1,9,21,62
FiveStars,378,1.25661,2.51895,0,0,0,1,16
lagged_srs,378,1.794,10.2467,-23.53,-5.345,2.17,8.7575,24.51
stateunemp,378,7.93989,1.92523,3.27,6.68,7.82,9.26,13.5
stadium_cap,378,56468,22394.3,20000,36096.8,55000,72000,109901
stadium_years_open,378,61.1693,27.0236,0,43,64,86,101
stadium_years_rennovation,378,7.0582,5.76432,0,3,6,10,27


### Summary stats, Power 5 teams

In [5]:
desp5 = p5.describe()
dp5 = desp5.reset_index()

dp52 = dp5.transpose()

dp52

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
TwoStars,244,17.4836,12.9133,0,8,15,23,68
ThreeStars,244,52.9508,12.8971,20,44,54,61.25,86
FourStars,244,21.373,15.9525,0,9,16,33,62
FiveStars,244,1.93443,2.92073,0,0,1,3,16
lagged_srs,244,6.19985,8.27634,-14.87,0.635,6.055,12.27,24.51
stateunemp,244,7.82201,1.937,3.27,6.4575,7.76,9.2825,12.58
stadium_cap,244,68544.7,17710.1,32740,55000,64150.5,82112,109901
stadium_years_open,244,69.9959,26.0137,1,53,83,89,101
stadium_years_rennovation,244,6.43852,4.97156,0,2,6,10,25


### Summary Stats, Non-Power 5 teams

In [6]:
desnonp5 = nonp5.describe()
dnonp5 = desnonp5.reset_index()

dnonp52 = dnonp5.transpose()

dnonp52

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
TwoStars,134,62.3209,14.0534,17,54,66,71,96
ThreeStars,134,28.0896,13.3706,6,19,25,35,69
FourStars,134,0.992537,1.75789,0,0,0,1,9
FiveStars,134,0.0223881,0.148497,0,0,0,0,1
lagged_srs,134,-6.22858,8.48596,-23.53,-11.7175,-6.155,-0.3025,18.69
stateunemp,134,8.15455,1.89193,4.13,6.805,7.83,9.0975,13.5
stadium_cap,134,34477.7,9681.36,20000,30000,30850,38019,65857
stadium_years_open,134,45.097,20.7469,0,39,45,59.5,82
stadium_years_rennovation,134,8.18657,6.85858,0,3,6,12,27


### Two Stage Least Squares



### 2SLS #1: Excluding *num_jcs*

In [7]:
dependent = df.ResponsiveRevThous
exog = var[['lagged_srs','stateunemp','num_homegames','stadium_cap','stadium_years_open','stadium_years_rennovation','coach_years_at_school','coach_experience','conf_coy','nat_coy']]
endog = var[['ThreeStars','FourStars','FiveStars']]
instruments0 = var[['pop/d1prog','hs_grad_pct','pro_team_per_million','spring_practice']]

In [8]:
mod = IV2SLS(dependent, exog, endog, instruments0)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:     ResponsiveRevThous   R-squared:                      0.7914
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7840
No. Observations:                 378   F-statistic:                    1339.1
Date:                Thu, Sep 27 2018   P-value (F-stat)                0.0000
Time:                        16:44:08   Distribution:                 chi2(13)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
=============================================================================================
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
lagged_srs                    656.08     274.98     2.3859     0.0170      117.13      1195.0
stateunemp                   -2623.9     596.04    -4.4022     0.0000     -3792.1     -1455.7
num_homegames                 2921.1     2454.4     1.1901     0.2340     -1889.5      7731.6
stadium_cap                   0.8557     0.1677     5.1016     0.0000      0.5270      1.1845
stadium_years_open            46.936     30.234     1.5524     0.1206     -12.322      106.19
stadium_years_rennovation    -14.269     144.33    -0.0989     0.9212     -297.15      268.61
coach_years_at_school        -213.37     265.83    -0.8027     0.4222     -734.39      307.64
coach_experience              159.42     195.84     0.8140     0.4156     -224.42      543.27
conf_coy                      3218.7     1816.4     1.7720     0.0764     -341.33      6778.8
nat_coy                      -496.44     2778.7    -0.1787     0.8582     -5942.6      4949.7
ThreeStars                   -582.28     323.57    -1.7995     0.0719     -1216.5      51.913
FourStars                    -493.44     509.82    -0.9679     0.3331     -1492.7      505.79
FiveStars                     1149.5     2798.5     0.4108     0.6812     -4335.4      6634.4
=============================================================================================

Endogenous: ThreeStars, FourStars, FiveStars
Instruments: pop/d1prog, hs_grad_pct, pro_team_per_million, spring_practice
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x116d9cd68

### 2SLS #2: Excluding *pop/d1prog*

In [9]:
instruments1 = var[['spring_practice','num_jcs','hs_grad_pct','pro_team_per_million']]

mod = IV2SLS(dependent, exog, endog, instruments1)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:     ResponsiveRevThous   R-squared:                      0.8073
Estimator:                    IV-2SLS   Adj. R-squared:                 0.8004
No. Observations:                 378   F-statistic:                    1293.7
Date:                Thu, Sep 27 2018   P-value (F-stat)                0.0000
Time:                        16:44:11   Distribution:                 chi2(13)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
=============================================================================================
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
lagged_srs                    587.62     248.47     2.3650     0.0180      100.63      1074.6
stateunemp                   -1797.6     388.37    -4.6287     0.0000     -2558.8     -1036.4
num_homegames                 408.90     1438.0     0.2844     0.7761     -2409.5      3227.3
stadium_cap                   0.8000     0.1446     5.5326     0.0000      0.5166      1.0834
stadium_years_open            3.6808     30.865     0.1193     0.9051     -56.814      64.176
stadium_years_rennovation    -26.593     129.46    -0.2054     0.8373     -280.34      227.15
coach_years_at_school        -372.44     339.91    -1.0957     0.2732     -1038.6      293.77
coach_experience              144.15     190.19     0.7579     0.4485     -228.61      516.91
conf_coy                      2265.7     1701.9     1.3312     0.1831     -1070.0      5601.3
nat_coy                       4002.7     3705.4     1.0802     0.2800     -3259.8   1.127e+04
ThreeStars                   -290.31     170.86    -1.6991     0.0893     -625.18      44.565
FourStars                     128.39     655.40     0.1959     0.8447     -1156.2      1412.9
FiveStars                    -3392.7     4940.9    -0.6867     0.4923  -1.308e+04      6291.2
=============================================================================================

Endogenous: ThreeStars, FourStars, FiveStars
Instruments: spring_practice, num_jcs, hs_grad_pct, pro_team_per_million
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x118081978

### 2SLS #3: Excluding *spring_practice*

In [10]:
instruments2 = var[['pop/d1prog','num_jcs','hs_grad_pct','pro_team_per_million']]

mod = IV2SLS(dependent, exog, endog, instruments2)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:     ResponsiveRevThous   R-squared:                      0.8325
Estimator:                    IV-2SLS   Adj. R-squared:                 0.8266
No. Observations:                 378   F-statistic:                    1502.5
Date:                Thu, Sep 27 2018   P-value (F-stat)                0.0000
Time:                        16:44:13   Distribution:                 chi2(13)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
=============================================================================================
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
lagged_srs                    473.29     207.46     2.2813     0.0225      66.674      879.90
stateunemp                   -2034.4     331.17    -6.1429     0.0000     -2683.4     -1385.3
num_homegames                 201.47     1367.8     0.1473     0.8829     -2479.3      2882.2
stadium_cap                   0.7828     0.1296     6.0422     0.0000      0.5289      1.0368
stadium_years_open            29.280     20.347     1.4390     0.1501     -10.600      69.159
stadium_years_rennovation    -72.271     109.29    -0.6613     0.5084     -286.47      141.93
coach_years_at_school        -23.603     200.20    -0.1179     0.9061     -415.98      368.77
coach_experience              71.696     168.93     0.4244     0.6713     -259.41      402.80
conf_coy                      2532.1     1549.2     1.6345     0.1022     -504.19      5568.5
nat_coy                       0.2460     2425.7     0.0001     0.9999     -4754.1      4754.6
ThreeStars                   -171.55     136.15    -1.2600     0.2077     -438.40      95.307
FourStars                    -614.34     452.64    -1.3572     0.1747     -1501.5      272.83
FiveStars                     3089.9     2503.6     1.2341     0.2171     -1817.2      7996.9
=============================================================================================

Endogenous: ThreeStars, FourStars, FiveStars
Instruments: pop/d1prog, num_jcs, hs_grad_pct, pro_team_per_million
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x1180b57b8

### 2SLS #4: Excluding *hs_grad_pct*

In [11]:
instruments3 = var[['pop/d1prog','num_jcs','spring_practice','pro_team_per_million']]

mod = IV2SLS(dependent, exog, endog, instruments3)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:     ResponsiveRevThous   R-squared:                      0.8540
Estimator:                    IV-2SLS   Adj. R-squared:                 0.8488
No. Observations:                 378   F-statistic:                    1879.2
Date:                Thu, Sep 27 2018   P-value (F-stat)                0.0000
Time:                        16:44:16   Distribution:                 chi2(13)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
=============================================================================================
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
lagged_srs                    381.05     219.24     1.7381     0.0822     -48.648      810.76
stateunemp                   -2345.9     377.43    -6.2155     0.0000     -3085.7     -1606.2
num_homegames                 2773.3     1755.9     1.5794     0.1142     -668.14      6214.8
stadium_cap                   0.4536     0.2429     1.8675     0.0618     -0.0225      0.9297
stadium_years_open            4.6687     24.802     0.1882     0.8507     -43.943      53.281
stadium_years_rennovation     138.67     163.70     0.8471     0.3969     -182.17      459.52
coach_years_at_school        -211.80     212.89    -0.9949     0.3198     -629.06      205.45
coach_experience              189.77     159.49     1.1898     0.2341     -122.83      502.38
conf_coy                      2551.7     1486.0     1.7172     0.0859     -360.73      5464.1
nat_coy                      -1289.1     2337.9    -0.5514     0.5814     -5871.2      3293.0
ThreeStars                   -310.04     140.46    -2.2073     0.0273     -585.34     -34.745
FourStars                     605.06     880.23     0.6874     0.4918     -1120.2      2330.3
FiveStars                    -1958.0     3736.8    -0.5240     0.6003     -9281.9      5366.0
=============================================================================================

Endogenous: ThreeStars, FourStars, FiveStars
Instruments: pop/d1prog, num_jcs, spring_practice, pro_team_per_million
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x1180b52b0

### 2SLS #5: Excluding *pro_team_per_million*

In [12]:
instruments4 = var[['pop/d1prog','num_jcs','spring_practice','hs_grad_pct']]

mod = IV2SLS(dependent, exog, endog, instruments4)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:     ResponsiveRevThous   R-squared:                      0.7246
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7148
No. Observations:                 378   F-statistic:                    880.18
Date:                Thu, Sep 27 2018   P-value (F-stat)                0.0000
Time:                        16:44:20   Distribution:                 chi2(13)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
=============================================================================================
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
lagged_srs                    392.51     268.15     1.4638     0.1433     -133.05      918.08
stateunemp                   -1936.5     441.25    -4.3886     0.0000     -2801.3     -1071.7
num_homegames                -1264.2     1955.9    -0.6463     0.5181     -5097.7      2569.4
stadium_cap                   0.8457     0.1679     5.0381     0.0000      0.5167      1.1747
stadium_years_open            45.641     26.401     1.7287     0.0839     -6.1050      97.387
stadium_years_rennovation    -168.44     142.41    -1.1828     0.2369     -447.55      110.67
coach_years_at_school         262.51     342.49     0.7665     0.4434     -408.75      933.77
coach_experience             -20.835     238.75    -0.0873     0.9305     -488.77      447.10
conf_coy                      2506.7     1848.2     1.3563     0.1750     -1115.6      6129.1
nat_coy                      -1701.5     3304.6    -0.5149     0.6066     -8178.4      4775.3
ThreeStars                    29.168     224.49     0.1299     0.8966     -410.83      469.17
FourStars                    -1378.6     713.86    -1.9312     0.0535     -2777.7      20.521
FiveStars                     8448.6     4558.5     1.8534     0.0638     -485.85   1.738e+04
=============================================================================================

Endogenous: ThreeStars, FourStars, FiveStars
Instruments: pop/d1prog, num_jcs, spring_practice, hs_grad_pct
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x116db8cc0

### 2SLS #6: No exclusions

In [13]:
instruments5 = var[['pop/d1prog','num_jcs','spring_practice','hs_grad_pct','pro_team_per_million']]

mod = IV2SLS(dependent, exog, endog, instruments5)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:     ResponsiveRevThous   R-squared:                      0.8315
Estimator:                    IV-2SLS   Adj. R-squared:                 0.8255
No. Observations:                 378   F-statistic:                    1500.6
Date:                Thu, Sep 27 2018   P-value (F-stat)                0.0000
Time:                        16:44:23   Distribution:                 chi2(13)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
=============================================================================================
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
lagged_srs                    491.71     224.79     2.1874     0.0287      51.122      932.30
stateunemp                   -2025.0     327.96    -6.1746     0.0000     -2667.8     -1382.2
num_homegames                 184.33     1367.7     0.1348     0.8928     -2496.2      2864.9
stadium_cap                   0.8029     0.1338     5.9991     0.0000      0.5406      1.0653
stadium_years_open            29.516     20.520     1.4384     0.1503     -10.702      69.734
stadium_years_rennovation    -77.794     108.27    -0.7185     0.4724     -289.99      134.40
coach_years_at_school        -43.189     197.58    -0.2186     0.8270     -430.45      344.07
coach_experience              73.567     169.83     0.4332     0.6649     -259.29      406.42
conf_coy                      2538.9     1556.7     1.6310     0.1029     -512.15      5590.0
nat_coy                       300.58     2269.7     0.1324     0.8946     -4147.9      4749.1
ThreeStars                   -186.96     139.57    -1.3395     0.1804     -460.51      86.600
FourStars                    -623.60     460.59    -1.3539     0.1758     -1526.3      279.14
FiveStars                     2860.2     2357.1     1.2134     0.2250     -1759.7      7480.0
=============================================================================================

Endogenous: ThreeStars, FourStars, FiveStars
Instruments: pop/d1prog, num_jcs, spring_practice, hs_grad_pct, pro_team_per_million
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x118109588

# Fixed Effects Model

In [14]:
panelx = var[['lagged_srs','stateunemp','num_homegames','stadium_cap','stadium_years_open','stadium_years_rennovation','coach_years_at_school','coach_experience','conf_coy','nat_coy','ThreeStars','FourStars','FiveStars']]
panelx = sm.add_constant(panelx)
panely = var['ResponsiveRevThous']

mod = PanelOLS(panely, panelx, entity_effects=True)
fixed_effects = mod.fit()
print(fixed_effects)

                          PanelOLS Estimation Summary                           
Dep. Variable:     ResponsiveRevThous   R-squared:                        0.3183
Estimator:                   PanelOLS   R-squared (Between):              0.3375
No. Observations:                 378   R-squared (Within):               0.3183
Date:                Thu, Sep 27 2018   R-squared (Overall):              0.3343
Time:                        16:44:26   Log-likelihood                   -3889.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      10.342
Entities:                          77   P-value                           0.0000
Avg Obs:                       4.9091   Distribution:                  F(13,288)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             10.342
                            

# Power 5 Fixed Effects Model

In [15]:
panelx = p5[['lagged_srs','stateunemp','num_homegames','stadium_cap','stadium_years_open','stadium_years_rennovation','coach_years_at_school','coach_experience','conf_coy','nat_coy','ThreeStars','FourStars','FiveStars']]
panelx = sm.add_constant(panelx)
panely = p5['ResponsiveRevThous']

mod = PanelOLS(panely, panelx, entity_effects=True)
fixed_effects = mod.fit()
print(fixed_effects)

                          PanelOLS Estimation Summary                           
Dep. Variable:     ResponsiveRevThous   R-squared:                        0.4503
Estimator:                   PanelOLS   R-squared (Between):             -12.916
No. Observations:                 244   R-squared (Within):               0.4503
Date:                Thu, Sep 27 2018   R-squared (Overall):             -8.9938
Time:                        16:44:30   Log-likelihood                   -2534.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      11.342
Entities:                          76   P-value                           0.0000
Avg Obs:                       3.2105   Distribution:                  F(13,180)
Min Obs:                       0.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             11.342
                            

# Non Power 5 Fixed Effects

In [16]:
panelx = nonp5[['lagged_srs','stateunemp','num_homegames','stadium_cap','stadium_years_open','stadium_years_rennovation','coach_years_at_school','coach_experience','conf_coy','nat_coy','ThreeStars','FourStars','FiveStars']]
panelx = sm.add_constant(panelx)
panely = nonp5['ResponsiveRevThous']

mod = PanelOLS(panely, panelx, entity_effects=True)
fixed_effects = mod.fit()
print(fixed_effects)

                          PanelOLS Estimation Summary                           
Dep. Variable:     ResponsiveRevThous   R-squared:                        0.3715
Estimator:                   PanelOLS   R-squared (Between):              0.0990
No. Observations:                 134   R-squared (Within):               0.3715
Date:                Thu, Sep 27 2018   R-squared (Overall):              0.1239
Time:                        16:44:33   Log-likelihood                   -1177.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      4.1824
Entities:                          77   P-value                           0.0000
Avg Obs:                       1.7403   Distribution:                   F(13,92)
Min Obs:                       0.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             4.1824
                            

### 2SLS, P5

In [17]:
dependent = p5.ResponsiveRevThous
exog = p5[['lagged_srs','stateunemp','num_homegames','stadium_cap','stadium_years_open','stadium_years_rennovation','coach_years_at_school','coach_experience','conf_coy','nat_coy']]
endog = p5[['ThreeStars','FourStars','FiveStars']]
instruments = p5[['pop/d1prog','num_jcs','spring_practice','hs_grad_pct','pro_team_per_million']]

mod = IV2SLS(dependent, exog, endog, instruments)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:     ResponsiveRevThous   R-squared:                      0.8729
Estimator:                    IV-2SLS   Adj. R-squared:                 0.8657
No. Observations:                 244   F-statistic:                    1963.7
Date:                Thu, Sep 27 2018   P-value (F-stat)                0.0000
Time:                        16:44:36   Distribution:                 chi2(13)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
=============================================================================================
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
lagged_srs                    395.99     310.74     1.2743     0.2025     -213.05      1005.0
stateunemp                   -2625.0     382.50    -6.8626     0.0000     -3374.7     -1875.3
num_homegames                -1676.9     1457.7    -1.1504     0.2500     -4533.8      1180.1
stadium_cap                   0.7482     0.1477     5.0656     0.0000      0.4587      1.0377
stadium_years_open           -31.630     28.593    -1.1062     0.2686     -87.671      24.411
stadium_years_rennovation     114.46     136.48     0.8387     0.4017     -153.03      381.95
coach_years_at_school        -225.11     218.41    -1.0307     0.3027     -653.18      202.96
coach_experience              188.49     223.48     0.8434     0.3990     -249.53      626.50
conf_coy                      2174.8     2102.5     1.0343     0.3010     -1946.2      6295.7
nat_coy                      -15.109     2598.9    -0.0058     0.9954     -5108.8      5078.5
ThreeStars                    142.88     145.52     0.9819     0.3262     -142.33      428.09
FourStars                    -159.06     611.57    -0.2601     0.7948     -1357.7      1039.6
FiveStars                     1908.2     2661.6     0.7170     0.4734     -3308.3      7124.8
=============================================================================================

Endogenous: ThreeStars, FourStars, FiveStars
Instruments: pop/d1prog, num_jcs, spring_practice, hs_grad_pct, pro_team_per_million
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x11818c748

### 2SLS, nonP5

In [18]:
dependent = nonp5.ResponsiveRevThous
exog = nonp5[['lagged_srs','stateunemp','num_homegames','stadium_cap','stadium_years_open','stadium_years_rennovation','coach_years_at_school','coach_experience','conf_coy','nat_coy']]
endog = nonp5[['ThreeStars','FourStars','FiveStars']]
instruments = nonp5[['pop/d1prog','num_jcs','spring_practice','hs_grad_pct','pro_team_per_million']]

mod = IV2SLS(dependent, exog, endog, instruments)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:     ResponsiveRevThous   R-squared:                      0.6470
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6091
No. Observations:                 134   F-statistic:                    284.41
Date:                Thu, Sep 27 2018   P-value (F-stat)                0.0000
Time:                        16:44:39   Distribution:                 chi2(13)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
=============================================================================================
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
lagged_srs                    217.46     64.973     3.3469     0.0008      90.113      344.80
stateunemp                   -477.60     201.75    -2.3673     0.0179     -873.02     -82.174
num_homegames                 1253.7     565.21     2.2181     0.0266      145.88      2361.5
stadium_cap                   0.2364     0.0527     4.4863     0.0000      0.1331      0.3397
stadium_years_open           -25.781     14.159    -1.8209     0.0686     -53.532      1.9695
stadium_years_rennovation    -58.682     59.934    -0.9791     0.3275     -176.15      58.786
coach_years_at_school         141.60     119.85     1.1815     0.2374     -93.300      376.51
coach_experience             -141.91     80.989    -1.7522     0.0797     -300.65      16.826
conf_coy                     -701.35     847.87    -0.8272     0.4081     -2363.2      960.45
nat_coy                       825.87     2042.0     0.4044     0.6859     -3176.4      4828.1
ThreeStars                   -175.38     93.307    -1.8796     0.0602     -358.26      7.4999
FourStars                     649.60     927.40     0.7005     0.4836     -1168.1      2467.3
FiveStars                    -5885.9   1.17e+04    -0.5032     0.6148  -2.881e+04   1.704e+04
=============================================================================================

Endogenous: ThreeStars, FourStars, FiveStars
Instruments: pop/d1prog, num_jcs, spring_practice, hs_grad_pct, pro_team_per_million
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x1181a71d0

In [22]:
d2.to_csv("summ.csv")

In [23]:
dp52.to_csv("p5summ.csv")

In [24]:
dnonp52.to_csv("nonp5summ.csv")

In [19]:
var.columns

Index(['TwoStars', 'ThreeStars', 'FourStars', 'FiveStars', 'lagged_srs',
       'stateunemp', 'stadium_cap', 'stadium_years_open',
       'stadium_years_rennovation', 'num_homegames', 'coach_experience',
       'conf_coy', 'nat_coy', 'coach_years_at_school', 'pop/d1prog',
       'hs_grad_pct', 'pro_team_per_million', 'spring_practice', 'num_jcs',
       'power', 'ResponsiveRevThous'],
      dtype='object')

In [20]:
from scipy import stats

transform = var['ResponsiveRevThous']
x = stats.boxcox(transform)

x

(array([41.6216774 , 42.42876398, 42.95907909, 44.11686143, 46.69164689,
        32.30945111, 32.43286078, 32.12983876, 32.59081848, 43.72334162,
        35.52691709, 34.84455927, 35.9232917 , 35.82183925, 38.83250544,
        38.9101291 , 41.62510788, 42.29301013, 42.03856239, 42.33040927,
        13.4778739 , 13.18572657, 15.913235  , 17.47977332, 21.12043862,
        42.43009507, 43.95493197, 43.97836152, 43.59893351, 46.19895732,
        29.31601234, 30.05469272, 30.43016081, 35.79682024, 32.0114099 ,
        32.79120225, 30.67595444, 33.98174713, 40.36066427, 19.01152003,
        18.03861502, 17.48111984, 20.88272176, 18.83353061, 35.00206941,
        35.46495327, 35.69739205, 36.11219171, 40.40223988, 33.78170238,
        33.64661713, 33.8860519 , 32.63600948, 34.8002934 , 20.50112904,
        21.13724809, 21.51643318, 20.67334549, 28.31354382, 25.19218475,
        28.54860909, 27.31597623, 25.27447465, 29.97959329, 26.8688799 ,
        28.38505537, 28.51136721, 27.84179694, 30.7

In [24]:
xx = stats.boxcox(transform, lmbda = 0)

xx

array([10.70571936, 10.78636757, 10.83861038, 10.9506738 , 11.1906431 ,
        9.65774378,  9.67328955,  9.63502526,  9.69311202, 10.91288564,
       10.04701886,  9.96713199, 10.09280388, 10.08112788, 10.41576157,
       10.42407982, 10.70606512, 10.77289951, 10.74755067, 10.77661374,
        6.33623618,  6.26050074,  6.92386482,  7.26615044,  7.97625847,
       10.78649944, 10.93516139, 10.93740908, 10.90087526, 11.14566942,
        9.26386664,  9.36419652,  9.41437849, 10.07824398,  9.61998487,
        9.71813804,  9.44693931,  9.86411976, 10.57685365,  7.57818077,
        7.38242364,  7.26643411,  7.93304876,  7.54297616,  9.98569426,
       10.03982067, 10.06676542, 10.11446653, 10.58115958,  9.83990782,
        9.82348674,  9.85255319,  9.69876768,  9.96190202,  7.86283894,
        7.97929883,  8.04736139,  7.89465619,  9.12416268,  8.66027553,
        9.15729881,  8.98086092,  8.67311619,  9.35409425,  8.91517826,
        9.13426829,  9.15206465,  9.05694394,  9.46037206, 10.88

In [25]:
shapiro = stats.shapiro(xx)

In [26]:
shapiro

(0.9526185393333435, 1.1499772245571194e-09)